# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 7 2023 8:30AM,256767,10,0086359340,ISDIN Corporation,Released
1,Feb 7 2023 8:30AM,256767,10,0086359205,ISDIN Corporation,Released
2,Feb 7 2023 8:30AM,256777,10,9123627,ISDIN Corporation,Released
3,Feb 7 2023 8:30AM,256767,10,0086359333,ISDIN Corporation,Released
4,Feb 7 2023 8:30AM,256767,10,0086359360,ISDIN Corporation,Released
5,Feb 7 2023 8:30AM,256767,10,0086359387,ISDIN Corporation,Released
6,Feb 7 2023 8:30AM,256767,10,0086359388,ISDIN Corporation,Released
7,Feb 7 2023 8:30AM,256764,10,0086359593,ISDIN Corporation,Released
8,Feb 7 2023 8:30AM,256764,10,0086359592,ISDIN Corporation,Released
9,Feb 7 2023 8:30AM,256764,10,0086359594,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,256770,Released,1
22,256771,Released,1
23,256777,Released,1
24,256778,Released,1
25,256781,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256770,NaN,NaN,1.0
256771,NaN,NaN,1.0
256777,NaN,NaN,1.0
256778,NaN,NaN,1.0
256781,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256676,0.0,0.0,2.0
256684,0.0,0.0,1.0
256685,0.0,1.0,0.0
256686,0.0,1.0,0.0
256695,32.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256676,0,0,2
256684,0,0,1
256685,0,1,0
256686,0,1,0
256695,32,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256676,0,0,2
1,256684,0,0,1
2,256685,0,1,0
3,256686,0,1,0
4,256695,32,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256676,,,2
1,256684,,,1
2,256685,,1,
3,256686,,1,
4,256695,32,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation
2,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation
7,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation
19,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation
20,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc."
21,Feb 7 2023 8:30AM,256761,10,Bio-PRF
31,Feb 6 2023 3:15PM,256771,22,"NBTY Global, Inc."
32,Feb 6 2023 3:11PM,256770,22,"NBTY Global, Inc."
33,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd"
54,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation,,,14
1,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation,,,1
2,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation,,,7
3,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation,,,1
4,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc.",,,1
5,Feb 7 2023 8:30AM,256761,10,Bio-PRF,,,7
6,Feb 6 2023 3:15PM,256771,22,"NBTY Global, Inc.",,,1
7,Feb 6 2023 3:11PM,256770,22,"NBTY Global, Inc.",,,1
8,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd",6,12,3
9,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,,,20


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation,14,,
1,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation,1,,
2,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation,7,,
3,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation,1,,
4,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc.",1,,
5,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7,,
6,Feb 6 2023 3:15PM,256771,22,"NBTY Global, Inc.",1,,
7,Feb 6 2023 3:11PM,256770,22,"NBTY Global, Inc.",1,,
8,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd",3,12,6
9,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation,14,,
1,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation,1,,
2,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation,7,,
3,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation,1,,
4,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation,14.0,NaN,NaN
1,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation,1.0,NaN,NaN
2,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation,7.0,NaN,NaN
3,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation,1.0,NaN,NaN
4,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation,14.0,0.0,0.0
1,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation,1.0,0.0,0.0
2,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation,7.0,0.0,0.0
3,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation,1.0,0.0,0.0
4,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2567515,108.0,0.0,2.0
15,513430,6.0,0.0,52.0
16,770139,2.0,1.0,0.0
19,770176,6.0,12.0,6.0
21,513369,1.0,1.0,0.0
22,513541,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2567515,108.0,0.0,2.0
1,15,513430,6.0,0.0,52.0
2,16,770139,2.0,1.0,0.0
3,19,770176,6.0,12.0,6.0
4,21,513369,1.0,1.0,0.0
5,22,513541,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,108.0,0.0,2.0
1,15,6.0,0.0,52.0
2,16,2.0,1.0,0.0
3,19,6.0,12.0,6.0
4,21,1.0,1.0,0.0
5,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,108.0
1,15,Released,6.0
2,16,Released,2.0
3,19,Released,6.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,2.0,52.0,0.0,6.0,0.0,0.0
Executing,0.0,0.0,1.0,12.0,1.0,0.0
Released,108.0,6.0,2.0,6.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,2.0,52.0,0.0,6.0,0.0,0.0
1,Executing,0.0,0.0,1.0,12.0,1.0,0.0
2,Released,108.0,6.0,2.0,6.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,2.0,52.0,0.0,6.0,0.0,0.0
1,Executing,0.0,0.0,1.0,12.0,1.0,0.0
2,Released,108.0,6.0,2.0,6.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()